Start Spark Session

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("WalmartStock").getOrCreate()

df = spark.read.csv("walmart_stock.csv", inferSchema = True, header = True)

Look at data structure and top 5 rows

In [2]:
df.printSchema()
df.show(5)

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)

+----------+------------------+---------+---------+------------------+--------+------------------+
|      Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+----------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|59.549999|

Basic stats of attributes, format numbers to 2 decimals

In [9]:
from pyspark.sql.functions import format_number

describe_df = df.describe()

describe_df.select("summary",
                    format_number(describe_df["Open"].cast("float"),2).alias("Open"),
                    format_number(describe_df["High"].cast("float"),2).alias("High"),
                    format_number(describe_df["Low"].cast("float"),2).alias("Low"),
                    format_number(describe_df["Close"].cast("float"),2).alias("Close"),
                    format_number(describe_df["Volume"].cast("float"),2).alias("Volume"),
                    ).show()

+-------+--------+--------+--------+--------+-------------+
|summary|    Open|    High|     Low|   Close|       Volume|
+-------+--------+--------+--------+--------+-------------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|     1,258.00|
|   mean|   72.36|   72.84|   71.92|   72.39| 8,222,093.50|
| stddev|    6.77|    6.77|    6.74|    6.76| 4,519,781.00|
|    min|   56.39|   57.06|   56.30|   56.42| 2,094,900.00|
|    max|   90.80|   90.97|   89.25|   90.47|80,898,096.00|
+-------+--------+--------+--------+--------+-------------+



Calculate HV ratio

In [12]:
df2 = df.withColumn("HV Ratio", df["High"]/df["Volume"])
df2.select("Date", "HV Ratio").show(5)

+----------+--------------------+
|      Date|            HV Ratio|
+----------+--------------------+
|2012-01-03|4.819714653321546E-6|
|2012-01-04|6.290848613094555E-6|
|2012-01-05|4.669412994783916E-6|
|2012-01-06|7.367338463826307E-6|
|2012-01-09|8.915604778943901E-6|
+----------+--------------------+
only showing top 5 rows



Day of peak high

In [18]:
df.orderBy(df["High"].desc()).head(1)[0][0]

'2015-01-13'

In [20]:
# Mean of Close
# Min and Max of Volume

from pyspark.sql.functions import mean, min, max
df.select(mean("Close")).show()

df.select(min("Volume"), max("Volume")).show()


+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+

+-----------+-----------+
|min(Volume)|max(Volume)|
+-----------+-----------+
|    2094900|   80898100|
+-----------+-----------+



In [27]:
# Number of days close was lower than 60
df.filter(df["Close"] < 60).count()

# Percentage of time the High was greater than 80 dollars
cnt = df.filter(df["High"] > 80).count()
cnt/df.count()*100

9.141494435612083

In [29]:
# Pearson correlation between High and Volume
from pyspark.sql.functions import corr
df.select(corr("High", "Volume")).show()

+-------------------+
| corr(High, Volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+



In [31]:
# max high per year
from pyspark.sql.functions import year
year_df = df.withColumn("Year", year(df["Date"]))
year_df.groupBy("Year").max().select("Year", "max(High)").show()

+----+---------+
|Year|max(High)|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+

